# Modelowanie procesu ryzyka

In [2]:
%run "C:\Users\ndzad\Dropbox\MINI\Magisterka\repo\mgr\Modelowanie_liczby_roszczen.ipynb"

      fun: 1375.1579339596738
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 4.54747313e-05, -2.27373677e-05, -2.51702660e-02,  4.54747313e-04])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 255
      nit: 42
     njev: 51
   status: 0
  success: True
        x: array([14.86573535,  6.34683826,  1.05347731,  8.96354863])
1     17
2     28
3     29
4     25
5     11
6     18
7     12
8     26
9     26
10    16
11     7
12    10
13    13
14    13
15    13
16     9
17    12
18    22
19    13
20    10
21     9
22    16
23    20
24    20
25    15
26    25
27    21
28    32
29     4
30    10
31     8
32     1
33     2
Name: Dis No, dtype: int64
Year
1     14
2     12
3     12
4     18
5     17
6     10
7     19
8     18
9     15
10    12
11    18
12    12
13    19
14    20
15    16
16    15
17    11
18     8
19    13
20    17
21    11
22    15
23     8
24    16
25    23
26    13
27    16
28    15
29    20
30     8
31    21
32    14
33    

In [3]:
%run "C:\Users\ndzad\Dropbox\MINI\Magisterka\repo\mgr\Modelowanie_wartosci_odszkodowan.ipynb"

1.395177366818116e-06
12.621668063027776 2.3407817241437945
1.0 1.0
-0.07922893559021643 275203.25662999536


C:\Users\ndzad\AppData\Local\Temp/ipykernel_7128/3568942317.py:5: RuntimeWarning: divide by zero encountered in log
  eq2 = np.log(np.mean(X)/np.power(np.prod(X),1/n))-np.log(alpha)+psi(alpha)
c:\Users\ndzad\anaconda3\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [73]:
def risk_process(mu, theta, X_distribution, X_params, lambda_func, lambda_fun_params, t):
    """
    Funkcja generuje proces ryzyka dla zadanych parametrów.

    Argumenty:
        mu: float
            Kapitał początkowy
        theta: float
            Narzut
        X_distribution: string
            Rozkład z jakiego pochodzą wartości odzkodowań
        X_params:
            Parametry rozkładu z jakiego pochodzą wartości odszkodowań
        lambda_func: funkcja
            Funkcja intensywności
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności
        t: int
            Maksymalna liczba lat
    Zwraca:
        numpy.ndarray
            Tablica procesu ryzyka.
    """ 
    n_pred = nhpp(lambda_func,lambda_fun_params,t+1)
    unique, counts = np.unique(np.ceil(n_pred[1]), return_counts=True)
    values = pd.DataFrame({'Year':unique, 'n':counts})
    index_df = pd.DataFrame({'Year':np.array(range(1,t+1))})
    merged_table = pd.merge(index_df, values, on='Year', how='left').fillna(0).set_index('Year')
    n_pred = pd.Series(merged_table['n']).astype(int)
    x = []
    y = []
    if X_distribution == 'expon':
        for i in range(1,t+1):
            x.append(sum(stats.expon.rvs(scale=1/X_params[0],size=n_pred[i])))
    S = np.cumsum(x)
    for i in range(0, t+1):
        y.append(sin_int_fun(i,lambda_fun_params)[0])
    Y = np.cumsum(y)[1:t+1]
    c = (1+theta)*mu*Y
    R = mu + c - S
    return R

In [101]:
risk_process(1e6,0.3,'expon',[1.395177366818116e-06],sin_int_fun,[14.86573535,  6.34683826,  1.05347731,  8.96354863],10)

array([3.79656171e+07, 5.38175804e+07, 7.30282955e+07, 9.23623859e+07,
       1.12151079e+08, 1.31002660e+08, 1.40313189e+08, 1.45992273e+08,
       1.60581722e+08, 1.66333296e+08])